# Feature Selection

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.pipeline import Pipeline

from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression

from sklearn.ensemble import RandomForestClassifier

from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import SelectFromModel

from sklearn.decomposition import PCA

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate
from sklearn.metrics import accuracy_score

In [ ]:
rng = np.random.RandomState(2)

In [ ]:
def grid_search(preprocessor, predictor, param_grid, cv, metric, X_train, y_train):
    model = Pipeline([
                ("preprocessor", preprocessor),
                ("predictor", predictor)
    ])

    gs = GridSearchCV(model, param_grid, scoring=metric, cv=cv, n_jobs=-1)

    gs.fit(X_train, y_train)

    return gs

In [ ]:
def check_fit(model, X_train, y_train, cv, metric):
    scores = cross_validate(model, X_train, y_train, cv=cv, scoring=metric, return_train_score=True, n_jobs=-1)
    return scores["train_score"].mean(), scores["test_score"].mean()

## Read in dataset, and split it

In [ ]:
import os
if 'google.colab' in str(get_ipython()):
    from google.colab import drive
    drive.mount('/content/drive')
    base_dir = "./drive/My Drive/Colab Notebooks/" # You may need to change this, depending on where your notebooks are on Google Drive
else:
    base_dir = "."
dataset_dir = os.path.join(base_dir, "datasets")

In [ ]:
df = pd.read_csv(os.path.join(dataset_dir, "glass.csv"))

In [ ]:
# A quick look at what it contains

df.head()

Each row contains a unique id and then the refractive index and oxide content of a sheet of glass. We have a multiclass classification problem - to predict the Type. The Types have already been encoded as integers, so we do not need to do Label Encoding. They are as follows:
- 1 float-processed glass for building windows
- 2 non-float processed glass for building windows
- 3 float-processed glass for vehicle windows
- 4 non-float-processed glass for vehicle windows (none in this dataset)
- 5 glass for containers
- 6 glass for tableware
- 7 glass for headlamps

In [ ]:
df.drop("id", axis=1, inplace=True)

In [ ]:
df.shape

Since there are 10 features but just 214 rows, there is a likelihood of overfitting.

In [ ]:
train, test = train_test_split(df, test_size=0.2, stratify=df["Type"], random_state=rng)

In [ ]:
features = np.array(["RI", "Na", "Mg", "Al", "Si", "K", "Ca", "Ba", "Fe"])

X_train = train[features]
y_train = train["Type"]
X_test = test[features]
y_test = test["Type"]

## Logistic Regression

In [ ]:
logistic_model = Pipeline([
        ("preprocessor", StandardScaler()),
        ("predictor", LogisticRegression(penalty=None, max_iter=800, random_state=rng))
    ])

logistic_model.fit(X_train, y_train)

In [ ]:
check_fit(logistic_model, X_train, y_train, cv=5, metric="accuracy")

Overfitting! (Bear in mind this is accuracy, not error, so you have to turn your reasoning upside down.)

## Regularization

We discussed regularization of Linear Regression: Lasso Regression uses the $l_1$-norm; Ridge regression uses the $l_2$-norm. There is a hyperparameter $\lambda$ to control the amount of regularization. scikit-learn has classes called `Lasso` and `Ridge` and it refers to $\lambda$ as `alpha`! Increasing the value of `alpha` increases the amount of regularization.

However, we are treating the prediction of wine quality as a multi-class classification problem, for which we would use Logistic Regression. But we can still regularize in a similar way. In scikit-learn, instead of having separate classes, the `LogisticRegression` class has an argument called `penalty`, whose values can be `None`, `l1`, `l2` (default) or `elasticnet` (which combines $l_1$ and $l_2$ regularization). One nuance is that, instead of `alpha`, we have `C`! Another nuance is that  `C` is the inverse of `alpha`: decreasing `C` increases the amount of regularization. A third nuance is that you must specify a solver and the default (`lbfgs`) only works for $l_2$ regularization. A solver that works for both is `saga`. Finally, it may not converge unless you specify a lot of iterations (`max_iter`, default 100) and relax the tolerance that determnes when it should stop (`tol`, default 0.0001). You may even need to switch to using `SGDClassifier` so that you can play with even more arguments such as the initial learning rate (`eta0`). Geez!

Let's try it. Remember, we need to scale the data because Logistic Regression uses Gradient Descent.

In [ ]:
regularized_logistic_gs = grid_search(
    preprocessor = StandardScaler(), 
    predictor = LogisticRegression(solver="saga", max_iter=800, random_state=rng),
    param_grid = {
        "predictor__penalty": ["l1", "l2"],
        "predictor__C": np.linspace(0.1, 0.2, num=10)
    },
    cv = 5,
    metric = "accuracy",
    X_train = X_train,
    y_train = y_train
)

regularized_logistic_gs.best_params_, regularized_logistic_gs.best_score_

In [ ]:
check_fit(regularized_logistic_gs.best_estimator_, X_train, y_train, cv=5, metric="accuracy")

We've closed the gap!

## Sequential Feature Selection Algorithms

This greedily inserts or discards a feature at a time - the one that does most or least to improve validation accuracy. In this case, it does this until it has only three features. On larger datasets, it will be slow because it must get the validation accuracy of many models to make its numerous decisions.

First let's show what it does on its own.

In [ ]:
sfs_preprocessor = Pipeline([
    ("scaler", StandardScaler()),
    ("f_selector", SequentialFeatureSelector(estimator = LogisticRegression(penalty=None, max_iter=800, random_state=rng),
                                             direction = "forward",
                                             n_features_to_select = 4,
                                             cv = 5))])

In [ ]:
sfs_preprocessor.fit(X_train, y_train)

Which four features did it retain? 

The `get_support` method of the `SequentialFeatureSelector` gives us a Boolean array.

In [ ]:
f_selector = sfs_preprocessor.named_steps["f_selector"]
support = f_selector.get_support()
features[support]

So now let's make it the preprocessor in a pipeline.

In [ ]:
sfs_logistic_gs = grid_search(
    preprocessor = sfs_preprocessor, 
    predictor = LogisticRegression(penalty=None, max_iter=800, random_state=rng),
    param_grid = {},
    cv = 5,
    metric = "accuracy",
    X_train = X_train,
    y_train = y_train
)

sfs_logistic_gs.best_params_, sfs_logistic_gs.best_score_

In [ ]:
check_fit(sfs_logistic_gs.best_estimator_, X_train, y_train, cv=5, metric="accuracy")

That's a substantial reduction in the size of the gap.

Of course, the problem is: we don't know how many features to retain. Above, I arbitrarily chose 3.

We can decide with a grid search - but it will build an awful lot of models.

In [ ]:
sfs_logistic_gs = grid_search(
    preprocessor = sfs_preprocessor, 
    predictor = LogisticRegression(penalty=None, max_iter=800, random_state=rng),
    param_grid = {
        "preprocessor__f_selector__n_features_to_select": range(1, 9)
    },
    cv = 5,
    metric = "accuracy",
    X_train = X_train,
    y_train = y_train
)

sfs_logistic_gs.best_params_, sfs_logistic_gs.best_score_

It finds that 3 features gives better validation accuracy than 4. 

Which features?

In [ ]:
f_selector = sfs_logistic_gs.best_estimator_.named_steps["preprocessor"].named_steps["f_selector"]
support = f_selector.get_support()
features[support]

In [ ]:
check_fit(sfs_logistic_gs.best_estimator_, X_train, y_train, cv=5, metric="accuracy")

The gap with 4 features is about the same it was with 3.

So now you could delete all the other features and retrain your model on just these four. Training and inference might now use less memory and take less time and you would be overfitting less than you were.

Of course, the model still overfits. The obvious solution is: more training examples!

## Filter Methods

We use scikit-learn's `SelectKBest` class. We must supply the function that measures feature importance. Lots of functions are already available. The two mentioned on the slides are F-value for regression (called `f_regression` in scikit-learn) and the ANOVA F-value for classification (called `f_classif`).

In [ ]:
skb_preprocessor = Pipeline([
    ("scaler", StandardScaler()),
    ("f_selector", SelectKBest(score_func=f_classif, k=None))])

In [ ]:
skb_logistic_gs = grid_search(
    preprocessor = skb_preprocessor, 
    predictor = LogisticRegression(penalty=None, max_iter=800, random_state=rng),
    param_grid = {
        "preprocessor__f_selector__k": range(1, 9)
    },
    cv = 5,
    metric = "accuracy",
    X_train = X_train,
    y_train = y_train
)

skb_logistic_gs.best_params_, skb_logistic_gs.best_score_

In [ ]:
f_selector = skb_logistic_gs.best_estimator_.named_steps["preprocessor"].named_steps["f_selector"]
support = f_selector.get_support()
features[support]

In [ ]:
check_fit(skb_logistic_gs.best_estimator_, X_train, y_train, cv=5, metric="accuracy")

It leaves us with a big gap.

Instead of using a function to calculate feature importances, we can train a model that can output feature importances. We'll do it with a Random Forest.

We won't use grid search to choose the number of Decision Trees inside the forest nor the maximum depth of those trees. Instead, we'll just use some small numbers: 100 and 2, respectively. This is for speed, but also because we aren't interested in finding a great Random Forest - just a decent estimate of feature importances.

In [ ]:
sfm_preprocessor = Pipeline([
    ("scaler", StandardScaler()),
    ("f_selector", SelectFromModel(estimator=RandomForestClassifier(n_estimators=100, max_depth=2, random_state=rng), max_features=None))])

In [ ]:
sfm_logistic_gs = grid_search(
    preprocessor = sfm_preprocessor, 
    predictor = LogisticRegression(penalty=None, max_iter=800, random_state=rng),
    param_grid = {
        "preprocessor__f_selector__max_features": range(1, 9)
    },
    cv = 5,
    metric = "accuracy",
    X_train = X_train,
    y_train = y_train
)

sfm_logistic_gs.best_params_, sfm_logistic_gs.best_score_

In [ ]:
f_selector = sfm_logistic_gs.best_estimator_.named_steps["preprocessor"].named_steps["f_selector"]
support = f_selector.get_support()
features[support]

In [ ]:
check_fit(sfm_logistic_gs.best_estimator_, X_train, y_train, cv=5, metric="accuracy")

Not bad.

## Dimensionality Reduction using PCA

Let's use PCA first and explain it afterwards.

In [ ]:
pca_preprocessor = Pipeline([
    ("scaler", StandardScaler()),
    ("f_selector", PCA(n_components=None, random_state=rng))])

In [ ]:
pca_logistic_gs = grid_search(
    preprocessor = pca_preprocessor, 
    predictor = LogisticRegression(penalty=None, max_iter=800, random_state=rng),
    param_grid = {
        "preprocessor__f_selector__n_components": range(1, 10)
    },
    cv = 5,
    metric = "accuracy",
    X_train = X_train,
    y_train = y_train
)

pca_logistic_gs.best_params_, pca_logistic_gs.best_score_

In [ ]:
check_fit(pca_logistic_gs.best_estimator_, X_train, y_train, cv=5, metric="accuracy")

Pretty good.

Our grid search retained five components: five of the new features that replace the original features.

In the previous parts of this Notebook, we were able to see which features were retained. But, with PCA, the components are not the same as the original features. They are linear combinations of the original features:

In [ ]:
f_selector = pca_logistic_gs.best_estimator_.named_steps["preprocessor"].named_steps["f_selector"]
f_selector.components_

So the first component (new feature) is $0.56 \times \mathit{RI} + -0.22 \times \mathit{Na} + 0.13 \times \mathit{Mg} + -0.43 \times \mathit{Al} + -0.23 \times \mathit{Si} + -0.27 \times \mathit{K} + 0.50 \times \mathit{Ca} + -0.19 \times \mathit{Ba} + 0.18 \times \mathit{Fe}$.

Above, we used a grid search to find how many new features to retain (`n_components`). 

Here is an alternative. We train a model that retains all of them:

In [ ]:
pca_logistic = Pipeline([
    ("preprocessor", pca_preprocessor),
    ("predictor", LogisticRegression(penalty=None, max_iter=800, random_state=rng))
])

In [ ]:
pca_logistic.fit(X_train, y_train)

Then, we can see how much of the variance each new feature 'explains':

In [ ]:
f_selector = pca_logistic.named_steps["preprocessor"].named_steps["f_selector"]
f_selector.explained_variance_

Or, better, the percentage of the variance each 'explains':

In [ ]:
f_selector.explained_variance_ratio_

And we plot the explained variance ratio:

In [ ]:
fig = plt.figure() 
sns.lineplot(f_selector.explained_variance_ratio_)
plt.xlabel("components")
plt.ylabel("explained variance ratio")
plt.show()

Sometimes (but note here!), the plot falls steeply, then there is an 'elbow', after which the plot is flatter. The elbow gives the number of new features to retain.